In [1]:
import colorstamps
import matplotlib.pyplot as plt
import numpy as np
import os

# n-variate colormaps
This notebook generates colormaps that form 

The colormaps are generated from the ‘CAM02-LCD’ colorspace as defined in the package colorspacious (https://pypi.org/project/colorspacious/)

https://github.com/trygvrad/colorstamps/ is used for easy access to this colorspace and visualization

In practice, we are looking for vectors that evenly dissects the available colorspace

In [2]:
colorbar_even_dict = {}
versions = ['A', 'B', 'C', 'D']
radius = 37 # max saturation

# ADD

In [3]:
for n in range(4, 15, 2):
    for version, r, max_J in zip(versions, [90/(n*2), -90/n+360/n, 90/n, -90/n+360/n], [56, 56, 77, 77]):
        name = f'{n}VarAdd{version}'
        # the range in lightness is adjust so that both colorbars increment at the first level, more than two can be combined (reaches 111 for 3 colormaps)
        J = (max_J, 0.75) # range in lightness 
        
        cmaps = [colorstamps.stamps.get_var_J(l = 512, J = J, rot = r + i/n*360.0, r = radius)[256,:256][::-1] for i in range(n)] 
        cmaps = np.array(cmaps)
        for c in cmaps:
            c[:,:] -= c[0,:][np.newaxis,:]
        
        
        # adjust greyscale by slightly shifting colorbars
        n2 = n//2
        for i in range(256):
            for j in range(n2):
                factor = 1#i/255 #scale the correction with distance from origin, no need to adjust low values
                average_RGB = np.average([cmaps[j][i], cmaps[j+n2][i]])
                for k in range(3):
                    d = average_RGB - np.average([cmaps[j][i, k],cmaps[j+n2][i, k]]) # adjustment to that color to make mix grey 
                    for c in cmaps[j], cmaps[j+n2]:
                        c[i,k] += factor*d
        
        cmaps[cmaps>1] = 1
        cmaps[cmaps<0] = 0
        
        for i, c in enumerate(cmaps):
            colorbar_even_dict[name+str(i)] = np.copy(c)
            

# SUB

In [4]:

for n in range(4, 15, 2):
    for version, r, min_J in zip(versions, [90/n, -90/n+360/n, 90/n, -90/n+360/n], [97, 97, 77.6, 77.6]):
        name = f'{n}VarSub{version}'
        # the range in lightness is adjust so that both colorbars increment at the first level, more than two can be combined (reaches 111 for 3 colormaps)
        J = (min_J, 128) # range in lightness 
        
        cmaps = [colorstamps.stamps.get_var_J(l = 512, J = J, rot = r + i/n*360.0, r = radius)[256,:256][::-1] for i in range(n)] 
        cmaps = np.array(cmaps)
        
        # adjust origin to 1
        for c in cmaps:
            c[:,:] += 1-c[0,:][np.newaxis,:]
        
        # adjust greyscale by slightly shifting colorbars
        n2 = n//2
        for i in range(256):
            for j in range(n2):
                factor = 1#i/255 #scale the correction with distance from origin, no need to adjust low values
                average_RGB = np.average([cmaps[j][i], cmaps[j+n2][i]])
                for k in range(3):
                    d = average_RGB - np.average([cmaps[j][i, k],cmaps[j+n2][i, k]]) # adjustment to that color to make mix grey 
                    for c in cmaps[j], cmaps[j+n2]:
                        c[i,k] += factor*d
        
        cmaps[cmaps>1] = 1
        cmaps[cmaps<0] = 0
        
        for i, c in enumerate(cmaps):
            colorbar_even_dict[name+str(i)] = np.copy(c)

In [5]:
colorbar_even_dict.keys()

dict_keys(['4VarAddA0', '4VarAddA1', '4VarAddA2', '4VarAddA3', '4VarAddB0', '4VarAddB1', '4VarAddB2', '4VarAddB3', '4VarAddC0', '4VarAddC1', '4VarAddC2', '4VarAddC3', '4VarAddD0', '4VarAddD1', '4VarAddD2', '4VarAddD3', '6VarAddA0', '6VarAddA1', '6VarAddA2', '6VarAddA3', '6VarAddA4', '6VarAddA5', '6VarAddB0', '6VarAddB1', '6VarAddB2', '6VarAddB3', '6VarAddB4', '6VarAddB5', '6VarAddC0', '6VarAddC1', '6VarAddC2', '6VarAddC3', '6VarAddC4', '6VarAddC5', '6VarAddD0', '6VarAddD1', '6VarAddD2', '6VarAddD3', '6VarAddD4', '6VarAddD5', '8VarAddA0', '8VarAddA1', '8VarAddA2', '8VarAddA3', '8VarAddA4', '8VarAddA5', '8VarAddA6', '8VarAddA7', '8VarAddB0', '8VarAddB1', '8VarAddB2', '8VarAddB3', '8VarAddB4', '8VarAddB5', '8VarAddB6', '8VarAddB7', '8VarAddC0', '8VarAddC1', '8VarAddC2', '8VarAddC3', '8VarAddC4', '8VarAddC5', '8VarAddC6', '8VarAddC7', '8VarAddD0', '8VarAddD1', '8VarAddD2', '8VarAddD3', '8VarAddD4', '8VarAddD5', '8VarAddD6', '8VarAddD7', '10VarAddA0', '10VarAddA1', '10VarAddA2', '10VarAddA3

In [6]:
colorbar_dict = {}
# transfer even
for n in range(4, 9, 2):
    if n == 4:
        v = versions
    else:
        v = ['A', 'C']
    for version in v:
        for i in range(n):
            for typ in ['Add', 'Sub']:
                name = f'{n}Var{typ}{version}{i}'
                colorbar_dict[name]  = colorbar_even_dict[name]
            #print(name)
# transfer odd
for n in range(5, 8, 2):
    v = ['A', 'C']
    for version in v:
        for i in range(n):
            for typ in ['Add', 'Sub']:
                name = f'{2*n}Var{typ}{version}{i*2+1}'
                new_name = f'{n}Var{typ}{version}{i}'
                colorbar_dict[new_name]  = colorbar_even_dict[name]
            #print(new_name)
print(colorbar_dict.keys())

dict_keys(['4VarAddA0', '4VarSubA0', '4VarAddA1', '4VarSubA1', '4VarAddA2', '4VarSubA2', '4VarAddA3', '4VarSubA3', '4VarAddB0', '4VarSubB0', '4VarAddB1', '4VarSubB1', '4VarAddB2', '4VarSubB2', '4VarAddB3', '4VarSubB3', '4VarAddC0', '4VarSubC0', '4VarAddC1', '4VarSubC1', '4VarAddC2', '4VarSubC2', '4VarAddC3', '4VarSubC3', '4VarAddD0', '4VarSubD0', '4VarAddD1', '4VarSubD1', '4VarAddD2', '4VarSubD2', '4VarAddD3', '4VarSubD3', '6VarAddA0', '6VarSubA0', '6VarAddA1', '6VarSubA1', '6VarAddA2', '6VarSubA2', '6VarAddA3', '6VarSubA3', '6VarAddA4', '6VarSubA4', '6VarAddA5', '6VarSubA5', '6VarAddC0', '6VarSubC0', '6VarAddC1', '6VarSubC1', '6VarAddC2', '6VarSubC2', '6VarAddC3', '6VarSubC3', '6VarAddC4', '6VarSubC4', '6VarAddC5', '6VarSubC5', '8VarAddA0', '8VarSubA0', '8VarAddA1', '8VarSubA1', '8VarAddA2', '8VarSubA2', '8VarAddA3', '8VarSubA3', '8VarAddA4', '8VarSubA4', '8VarAddA5', '8VarSubA5', '8VarAddA6', '8VarSubA6', '8VarAddA7', '8VarSubA7', '8VarAddC0', '8VarSubC0', '8VarAddC1', '8VarSubC1', '

# Save

In [7]:
import pickle
with open(r"n_multivariate_cmaps.pickle", "wb") as output_file:
    pickle.dump(colorbar_dict, output_file)

In [8]:
for n in range(2, 9, 1):
    folder = f'comlormap_plots/{n}/'
    if not os.path.exists(folder):
        os.makedirs(folder)
    if n == 2:
        v = ['A','B']
    elif n < 5:
        v = versions
    else:
        v = ['A', 'C']
    for version in v:
        name = f'{n}VarSub{version}'
        cmaps = [colorbar_dict[f'{name}{i}'] for i in range(n)]
        # plot colorbars
        fig, axes = plt.subplots(4, n, figsize = (n*3, 12), dpi = 200, facecolor = 'w')
        axes = np.atleast_2d(axes)
        for i in range(n):
            axes[0,i].imshow(cmaps[i]*np.ones((256,256,3)))
            deuteranopia = colorspacious.cspace_convert(cmaps[i]*np.ones((256,256,3)), cvd_d, "sRGB1")
            deuteranopia[deuteranopia<0] = 0
            deuteranopia[deuteranopia>1] = 1
            axes[1,i].imshow(deuteranopia)
            axes[1,i].set_title('deuteranopia')
            protanomaly = colorspacious.cspace_convert(cmaps[i]*np.ones((256,256,3)), cvd_p, "sRGB1")
            protanomaly[protanomaly<0] = 0
            protanomaly[protanomaly>1] = 1
            axes[2,i].imshow(protanomaly)
            axes[2,i].set_title('protanomaly')
            tritanomaly = colorspacious.cspace_convert(cmaps[i]*np.ones((256,256,3)), cvd_t, "sRGB1")
            tritanomaly[tritanomaly<0] = 0
            tritanomaly[tritanomaly>1] = 1
            axes[3,i].imshow(tritanomaly)
            axes[3,i].set_title('tritanomaly')
        for ax in axes.ravel():
            ax.set_xticks([])
            ax.set_yticks([])
        fig.tight_layout()
        fig.savefig(folder+name+'cmaps.png')
        plt.close(fig)
        
        # make all 2D cmaps
        fig, axes = plt.subplots(n*(n-1)//2,5, figsize = (12,3*n*(n-1)//2), dpi = 100, facecolor = 'w')
        axes = np.atleast_2d(axes)
        k = 0
        for i in range(n):
            for j in range(i+1,n):
                cmap = 1 -(1 -cmaps[i][:,np.newaxis]) -(1 -cmaps[j][np.newaxis,:])
                cmap[cmap<0] = 0
                cmap[cmap>1] = 1
                colorstamps.Stamp(cmap[:,:]*(1-2e-5) + 1e-5).eval(axes[k]) # include a delta (+1e-5) to avoid error
                axes[k,0].set_title(f'{i} vs {j}')
                k += 1
        fig.tight_layout()
        print(folder+name+'compare.png', end = '\r')
        plt.close(fig)

KeyError: '2VarSubA0'